In [1]:
!pip install deepface pandas opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=629057e548589d8b7687f358fff0dfc3f380b887cbfd906960f79ae2e0d8843d
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [2]:
import pandas as pd
import cv2
import os
import numpy as np
from deepface import DeepFace
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
from tqdm import tqdm

24-11-29 17:36:29 - Directory /root/.deepface has been created
24-11-29 17:36:29 - Directory /root/.deepface/weights has been created


In [3]:
file_path = '/content/Data.csv'  # Update this with your file path in Colab
data = pd.read_csv(file_path)

In [4]:
print(data.head())

   Performance                                          Video URL
0       1.1060  https://fgimagestorage.blob.core.windows.net/f...
1       2.2447  https://fgimagestorage.blob.core.windows.net/f...
2       2.0126  https://fgimagestorage.blob.core.windows.net/f...
3       1.7708  https://fgimagestorage.blob.core.windows.net/f...
4       0.6293  https://fgimagestorage.blob.core.windows.net/f...


In [5]:
def extract_faces_from_video(video_path, output_folder="faces"):
    """
    Extracts faces from video frames and saves them.
    Args:
        video_path (str): Path to the video file.
        output_folder (str): Folder to save the extracted face images.
    Returns:
        List of file paths to extracted faces.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    frame_count = 0
    face_paths = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

        for i, (x, y, w, h) in enumerate(faces):
            face = frame[y:y+h, x:x+w]
            face_path = os.path.join(output_folder, f"frame_{frame_count}_face_{i}.jpg")
            cv2.imwrite(face_path, face)
            face_paths.append(face_path)

        frame_count += 1
        if frame_count > 50:  # Limit to first 50 frames for efficiency
            break

    cap.release()
    return face_paths

In [6]:
# Function to generate embeddings for faces
def generate_embeddings(face_paths):
    embeddings = {}
    for face_path in tqdm(face_paths):
        try:
            embedding = DeepFace.represent(img_path=face_path, model_name='Facenet')[0]['embedding']
            embeddings[face_path] = embedding
        except Exception as e:
            print(f"Error processing {face_path}: {e}")
    return embeddings

In [7]:
# Store the influencer's embeddings and performance tracking
unique_faces = {}
influencer_performance = {}
influencer_videos = {}

# Iterate through the dataset
for index, row in data.iterrows():
    video_url = row['Video URL']
    faces_in_video = extract_faces_from_video(video_url)
    embeddings = generate_embeddings(faces_in_video)

    for face_path, embedding in embeddings.items():
        assigned = False
        for influencer_id, ref_embedding in unique_faces.items():
            if cosine_similarity([embedding], [ref_embedding])[0][0] > 0.8:  # Threshold for similarity
                # Associate this video with the influencer
                if influencer_id not in influencer_videos:
                    influencer_videos[influencer_id] = []
                influencer_videos[influencer_id].append(video_url)

                influencer_performance[influencer_id].append(row['Performance'])
                assigned = True
                break

        if not assigned:
            # Assign a new influencer ID
            new_influencer_id = f"influencer_{len(unique_faces) + 1}"
            unique_faces[new_influencer_id] = embedding
            influencer_performance[new_influencer_id] = [row['Performance']]
            influencer_videos[new_influencer_id] = [video_url]

0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/2 [00:00<?, ?it/s]

24-11-29 17:38:14 - facenet_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: /root/.deepface/weights/facenet_weights.h5

  0%|          | 0.00/92.2M [00:00<?, ?B/s]
 12%|█▏        | 11.0M/92.2M [00:00<00:00, 82.6MB/s]
 23%|██▎       | 21.5M/92.2M [00:00<00:00, 77.7MB/s]
 39%|███▉      | 36.2M/92.2M [00:00<00:00, 104MB/s] 
 57%|█████▋    | 53.0M/92.2M [00:00<00:00, 103MB/s]
 71%|███████   | 65.5M/92.2M [00:00<00:00, 109MB/s]
100%|██████████| 92.2M/92.2M [00:00<00:00, 112MB/s]
100%|██████████| 2/2 [00:06<00:00,  3.02s/it]

Error processing faces/frame_39_face_0.jpg: Face could not be detected in faces/frame_39_face_0.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error processing faces/frame_43_face_0.jpg: Face could not be detected in faces/frame_43_face_0.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.



 88%|████████▊ | 43/49 [00:16<00:01,  3.23it/s]

...

 94%|█████████▍| 79/84 [00:12<00:00,  5.10it/s]

Error processing faces/frame_49_face_0.jpg: Face could not be detected in faces/frame_49_face_0.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


100%|██████████| 84/84 [00:13<00:00,  6.34it/s]


Error processing faces/frame_50_face_2.jpg: Face could not be detected in faces/frame_50_face_2.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


0it [00:00, ?it/s]


In [8]:

# Calculate the average performance for each influencer
average_performance = {influencer_id: np.mean(performance)
                       for influencer_id, performance in influencer_performance.items()}


In [9]:
# Prepare the output DataFrame to include video associations
output_data = []

for influencer_id in average_performance:
    videos = ", ".join(influencer_videos[influencer_id])  # Join video URLs
    avg_performance = average_performance[influencer_id]
    output_data.append([influencer_id, avg_performance, videos])


In [10]:
# Save the output to CSV
output_df = pd.DataFrame(output_data, columns=["Influencer ID", "Average Performance", "Videos Appeared In"])

output_df.to_csv("influencer_performance_with_videos.csv", index=False)
print("Output saved as influencer_performance_with_videos.csv")

Output saved as influencer_performance_with_videos.csv


In [11]:
print(output_df.head())

  Influencer ID  Average Performance  \
0  influencer_1             0.961439   
1  influencer_2             0.812070   
2  influencer_3             0.947692   
3  influencer_4             0.566381   
4  influencer_5             0.938930   

                                  Videos Appeared In  
0  https://fgimagestorage.blob.core.windows.net/f...  
1  https://fgimagestorage.blob.core.windows.net/f...  
2  https://fgimagestorage.blob.core.windows.net/f...  
3  https://fgimagestorage.blob.core.windows.net/f...  
4  https://fgimagestorage.blob.core.windows.net/f...  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
import shutil

# Create a zip file
shutil.make_archive('faces', 'zip', 'faces')

'/content/faces.zip'

In [14]:
from google.colab import files

# Download the zip file
files.download('faces.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>